# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the algebraic following equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



The implementation in Scala is as follows (we already give the generic version `List[A]`, rather than the implementation of `IntList`):

Note that the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. However, this forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times. The standard definition looks as follows:

In [ ]:
object StdDefinition{
}

We will stick to the former definition. Some examples of lists: 

In [ ]:
// The empty list


In [ ]:
// Non-empty list [1, 2, 3]


### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [ ]:
import scala.{List => IList}



How can we do that with out own lists? We define a smart constructor in the companion object using variadic arguments: 

In [ ]:
object List{
}

This allows us to write lists more easily:

Note that the smart constructor `apply` is defined recursively. Let's dive into recursion.

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [ ]:
// Using mutable variables



The recursive function is implemented as follows: 

In [ ]:
// Using recursive functions



Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [ ]:
// Using tail-recursive functions



We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [ ]:
// First, imperatively



In [ ]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [ ]:
// Imperatively


In [ ]:
// Tail-recursive


In [ ]:
// Plain recursive


### Using the standard `List` type

In [ ]:
import scala.collection.immutable.List

From now on, we will use the `List` type defined in the standard library of Scala. For the sake of comparison, let's re-implement the `length` function: 

In [ ]:
/*
def length[A](list: List[A]): Int = 
    list match {
        case Empty()           => 0
        case NonEmpty(_, tail) => 1 + length(tail)
    }
*/


### Unit testing with `scalatest`

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [ ]:
/*
assert(length(List()) == 0)
assert(length(List(1)) == 1)
assert(length(List(1,2,3,4)) == 4)
*/

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [ ]:
class TestSum extends FlatSpec with Matchers{
    "length" should "work" in {
    }
}

In [ ]:
// Recursively



In [ ]:
// With tail-recursion



### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [2]:
class TestProduct(product: List[Int] => Int) extends FlatSpec with Matchers{
    "product" should "work" in {
        product(List()) shouldBe 1
        product(List(15)) shouldBe 15
        product(List(1,2,3)) shouldBe 1*2*3 // 6
        product(List(0,1,2,5,6,7,2,3,2,3)) shouldBe 0 // 0*1*2*5*....
    }
}

defined class TestProduct

This is the common recursive implementation:

In [3]:
def product(list: List[Int]): Int = 
    list match {
        case Nil => 1 : Int
        case head :: tail => 
            head * product(tail) : Int 
    }

defined function product

It works as expected: 

In [4]:
run(new TestProduct(product))

cmd1$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [5]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 
            1
        case head :: tail => 
            if (head == 0) 
                0
            else 
                head * product(tail)
    }


defined function product

In [7]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 
            1
        case head :: tail if head == 0 =>
                0
        case head :: tail => 
                head * product(tail)
    }


defined function product

In [12]:
def product(list: List[Int]): Int =
    list match {
        case Nil => 
            1
        case 0 :: _ =>
            0
        case head :: tail => 
            head * product(tail)
    }


defined function product

In [13]:
run(new TestProduct(product))

cmd1$Helper$TestProduct:
product
- should work


A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [14]:
//class TestMember(member: Function2[List[String], String, Boolean]) 
class TestMember(member: (List[String], String) => Boolean) 
extends FlatSpec with Matchers{
    "member" should "work" in {
        member(List("1", "hola", "adios"), "hola") shouldBe true
        member(List("1", "hola", "adios"), "") shouldBe false
        member(List("1", "hola", "adios"), "pasa") shouldBe false
        member(List(), "hola") shouldBe false
    }
}

defined class TestMember

In [33]:
def member[A](l: List[A], s: A): Boolean = 
    l match {
        case Nil => false
        case head :: tail => 
            (head == s) || member(tail, s) : Boolean
    }


defined function member

In [49]:
def member[A](l: List[A], s: A): Boolean = 
    l match {
        case Nil => false
        case `s` :: tail => 
            true
        case _ :: tail => 
            member(tail, s) : Boolean
    }


defined function member

In [50]:
member[Int](List(1,2,3), 3)

res49: Boolean = true

In [51]:
member[Any](List(1: Any, "hola": Any, 'c': Any), 'c': Any)

res50: Boolean = true

In [52]:
member(List(1, "hola", 'c'), 'c')

res51: Boolean = true

In [53]:
def memberV[A]: (List[A], A) => Boolean = 
    (l: List[A], a: A) => 
        l match {
            case Nil => false : Boolean 
            case head :: tail => head == a || memberV(tail, a)
        }

defined function memberV

In [56]:
def memberV[A]: (List[A], A) => Boolean = 
    {
        case (Nil, a) => 
            false
        case (head :: tail, a) => 
            head == a || memberV(tail, a)
    }

defined function memberV

In [55]:
//run(new TestMember(member[String]))
run(new TestMember(member))
run(new TestMember(memberV))

cmd13$Helper$TestMember:
member
- should work
cmd13$Helper$TestMember:
member
- should work


We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [59]:
class TestLast(last: List[Int] => Option[Int]) 
extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List(1,3,2,5,2)) shouldBe Some(2)
        last(List(1)) shouldBe (Some(1 : Int) : Option[Int])
        last(List()) shouldBe None
    }
}

defined class TestLast

In [63]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => None : Option[A]
        case (head: A) :: tail => 
            if (tail == Nil) Some(head) //  : Option[A]
            else last(tail) : Option[A] 
    }

defined function last

In [63]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => None : Option[A]
        case (head: A) :: tail => 
            tail match {
                case Nil => Some(head) //  : Option[A]
                case _ => last(tail) : Option[A] 
            }
    }

defined function last

In [67]:
def last[A](l: List[A]): Option[A] = 
    l match {
        case Nil => 
            None : Option[A]
        case (head: A) :: Nil => 
            Some(head) //  : Option[A]
        //case _ :: h2 :: tail => 
        //  last(h2 :: tail) : Option[A] 
        //case _ :: (t@(h2 :: tail)) => 
          //last(t) : Option[A] 
        case _ :: tail => 
            last(tail) : Option[A] 
            
    }

defined function last

In [68]:
run(new TestLast(last))

cmd58$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [81]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends FlatSpec with Matchers{
    "insertLast" should "work for the empty list" in {
        insertLast(List(), 2) shouldBe List(2)
    }
    
    it should "work for non-empty list" in {
        insertLast(List(1,3,2,1,5), 9) shouldBe List(1,3,2,1,5,9)
        insertLast(List(1), 0) shouldBe List(1,0)
    }
}

defined class TestInsertLast

In [82]:
def insertLast(l: List[Int], e: Int): List[Int] = 
    l match {
        case Nil => ??? : List[Int]
        case head :: tail => 
            ??? : List[Int]
    }

defined function insertLast

In [87]:
def insertLast(l: List[Int], e: Int): List[Int] = 
    l match {
        case Nil => 
            List(e) // e :: Nil : List[Int]
        case head :: tail => 
            head :: insertLast(tail, e) : List[Int]
    }

defined function insertLast

In [91]:
def insertLast[A](l: List[A], e: A): List[A] = 
    l match {
        case Nil => 
            List(e) // e :: Nil : List[Int]
        case head :: tail => 
            head :: insertLast(tail, e) // : List[A]
    }

defined function insertLast

In [88]:
run(new TestInsertLast(insertLast))

cmd80$Helper$TestInsertLast:
insertLast
- should work for the empty list
- should work for non-empty list


### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)

### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [94]:
class TestReverse(reverse: List[Int] => List[Int]) 
extends FlatSpec with Matchers{
    "reverse" should "work" in {
        reverse(List(1,2,3)) shouldBe List(3,2,1)
        reverse(List()) shouldBe List()
        reverse(List(2,52,61,2)) shouldBe List(2,61,52,2)
        reverse(List(1,2,3,4)) shouldBe List(4,3,2,1)
        reverse(1 :: List(2,3,4)) shouldBe List(4,3,2) :+ 1
    }
}

defined class TestReverse

In [93]:
List(4,3,2) :+ 1

res92: List[Int] = List(4, 3, 2, 1)

In [103]:
// Recursively: Really inefficient 
def reverse[A](l: List[A]): List[A] = 
    l match {
        case Nil => Nil : List[A]
        case head :: tail => 
            reverse(tail) :+ head : List[A]
    }

defined function reverse

In [104]:
List.fill(10)('a')

res103: List[Char] = List('a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a')

In [105]:
reverse(List.fill(1000000)('a'))

: 

In [96]:
run(new TestReverse(reverse))

cmd93$Helper$TestReverse:
reverse
- should work


In [111]:
// Tail-recursive, efficiently
/*
1 :: 2 :: 3 :: Nil 

out = Nil 
1 :: Nil
2 :: 1 :: Nil
3 :: 2 :: 1 :: Nil
*/
def reverseTR[A](l: List[A]): List[A] = {
    def auxReverse(out: List[A], aux: List[A]): List[A] = 
        aux match {
            case Nil => out
            case head :: tail => 
                auxReverse(head :: out, tail)
        }
    
    auxReverse(List(), l)
}

defined function reverseTR

In [112]:
run(new TestReverse(reverseTR))

cmd93$Helper$TestReverse:
reverse
- should work
